In [2]:
!which python
!nvidia-smi

/c/Users/agkav/miniconda3/envs/PyTorch-Development/python
Fri Jul  5 16:20:05 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.99                 Driver Version: 555.99         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   40C    P8              1W /  125W |       0MiB /   8188MiB |      0%      Default |
|                                         |                        |              

In [3]:
!pip install matplotlib

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import os
from sklearn.model_selection import train_test_split

In [5]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from torchvision.transforms import functional as F
from PIL import Image

class CoconutTreeDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_name).convert("RGB")
        
        boxes = self.annotations.iloc[idx, 1].split()
        boxes = [float(coord) for coord in boxes]
        boxes = torch.tensor(boxes, dtype=torch.float32).reshape(-1, 4)
        
        labels = torch.ones((boxes.shape[0],), dtype=torch.int64)  # 1 for coconut_tree
        
        if self.transform:
            image, boxes = self.transform(image, boxes)
        
        target = {"boxes": boxes, "labels": labels}
        
        return image, target

class ComposeTransforms:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img, boxes):
        for t in self.transforms:
            img, boxes = t(img, boxes)
        return img, boxes

class Resize:
    def __init__(self, size):
        self.size = size

    def __call__(self, img, boxes):
        orig_size = torch.tensor([img.width, img.height, img.width, img.height])
        img = F.resize(img, self.size)
        scale_factor = torch.tensor([self.size[1], self.size[0], self.size[1], self.size[0]]) / orig_size
        boxes = boxes * scale_factor
        return img, boxes

class RandomHorizontalFlip:
    def __init__(self, prob=0.5):
        self.prob = prob

    def __call__(self, img, boxes):
        if torch.rand(1) < self.prob:
            img = F.hflip(img)
            boxes[:, [0, 2]] = img.width - boxes[:, [2, 0]]
        return img, boxes

class RandomVerticalFlip:
    def __init__(self, prob=0.5):
        self.prob = prob

    def __call__(self, img, boxes):
        if torch.rand(1) < self.prob:
            img = F.vflip(img)
            boxes[:, [1, 3]] = img.height - boxes[:, [3, 1]]
        return img, boxes

class RandomRotation:
    def __init__(self, degrees):
        self.degrees = degrees

    def __call__(self, img, boxes):
        angle = torch.empty(1).uniform_(-self.degrees, self.degrees).item()
        img = F.rotate(img, angle)
        # Bounding box rotation can be complex and might require more specific handling.
        return img, boxes

# Define transforms with data augmentation
transform = ComposeTransforms([
    Resize((224, 224)),
    RandomHorizontalFlip(),
    RandomVerticalFlip(),
    RandomRotation(10),
    transforms.ToTensor(),
])


In [12]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split

class CoconutTreeDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_name).convert("RGB")
        
        boxes = self.annotations.iloc[idx, 1].split()
        boxes = [float(coord) for coord in boxes]
        boxes = torch.tensor(boxes, dtype=torch.float32).reshape(-1, 4)
        
        labels = torch.ones((boxes.shape[0],), dtype=torch.int64)  # 1 for coconut_tree
        
        if self.transform:
            image = self.transform(image)
        
        target = {"boxes": boxes, "labels": labels}
        return image, target

# Define transforms with data augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

# Create dataset
full_dataset = CoconutTreeDataset(csv_file=r'D:\code\CoconutCount\data\train\mbb_noempties.csv', img_dir= r'D:\code\CoconutCount\data\train\jpegs', transform=transform)

# Split the data
train_idx, val_idx = train_test_split(range(len(full_dataset)), test_size=0.2, random_state=42)

# Create Subset objects
train_dataset = Subset(full_dataset, train_idx)
val_dataset = Subset(full_dataset, val_idx)

# Custom collate function
def collate_fn(batch):
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    return images, targets

# Create dataloaders with the custom collate function
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)

# Check a sample batch from train_loader
images_batch, targets_batch = next(iter(train_loader))
print(images_batch[0].shape, targets_batch[0]['boxes'].shape)



torch.Size([3, 224, 224]) torch.Size([3, 4])


csv_file=r'D:\code\CoconutCount\data\train\mbb_noempties.csv', img_dir= r'D:\code\CoconutCount\data\train\jpegs'